In [2]:
from aiida import load_profile, orm
load_profile()

Profile<uuid='827a33dde1bc4d498af8e022f9e9dece' name='mikibonacci'>

In [3]:
from aiida_atomistic.data.structure.structure import StructureData

### Initialization and immutability

#### Just the crystal structure:

In [4]:
from ase import Atoms

unit_cell = [[3.5, 0.0, 0.0], [0.0, 3.5, 0.0], [0.0, 0.0, 3.5]]

atoms = Atoms('LiLi', [[0.0, 0.0, 0.0],[1.5, 1.5, 1.5]], cell = [1,1,1])
atoms.set_cell(unit_cell, scale_atoms=False)
atoms.set_pbc([True,True,True])

structure = StructureData(ase=atoms)

print(structure.cell)
print(structure.sites)
print(structure.kinds)   # <== backward-compatibility
print(structure.pbc)     # <== backward-compatibility

[[3.5, 0.0, 0.0], [0.0, 3.5, 0.0], [0.0, 0.0, 3.5]]
[<Site: kind name 'Li' @ 0.0,0.0,0.0>, <Site: kind name 'Li' @ 1.5,1.5,1.5>]
[<Kind: name 'Li', symbol 'Li'>]
(True, True, True)


#### Adding also properties:

In [5]:
structure = StructureData(
        ase=atoms,
        properties={
                'pbc': {'value':[True,False,True]},
                },
        )

print(structure.cell)
print(structure.sites)
print(structure.pbc)

[[3.5, 0.0, 0.0], [0.0, 3.5, 0.0], [0.0, 0.0, 3.5]]
[<Site: kind name 'Li' @ 0.0,0.0,0.0>, <Site: kind name 'Li' @ 1.5,1.5,1.5>]
(True, True, True)


In [6]:
structure.properties.pbc

Pbc(parent=<StructureData: uuid: 14a2cce6-05fb-46dc-ac8e-416caa6b6770 (unstored)>, value=[True, False, True])

### Immutability

In [7]:
structure.properties.pbc.value=5

TypeError: "Pbc" is immutable and does not support item assignment

In [8]:
structure.properties =  0

AttributeError: After the initialization, `properties` is a read-only attribute

In [9]:
structure.properties.pbc = 4

NotImplementedError: After the initialization, `properties` and its attributes are read-only!

#### Trying to set some unsupported property:

In [10]:
structure = StructureData(
        ase=atoms,
        properties={
                'pbc': {'value':[True,False,True]},
                'unsupported_property':{} 
        })

NotImplementedError: Property 'unsupported_property' is not yet supported.
Supported properties are: {'pbc', 'custom'}

#### Type checking for the `properties` input and its content

In [11]:
structure = StructureData(
        ase=atoms,
        properties=[('pbc',{'value':[True,False,True]})])

ValueError: The `properties` input is not of the right type. Expected '<class 'dict'>', received '<class 'list'>'.

In [12]:
structure = StructureData(
        ase=atoms,
        properties={
                'pbc': [{'value':[True,False,True]}],
        })

ValueError: The 'pbc' value is not of the right type. Expected '<class 'dict'>', received '<class 'list'>'.

# TESTS

In [19]:
structure = StructureData(
        ase=atoms,
        properties={
                'pbc': {'value':[True,False,True]},
                },
        )

assert structure.properties.pbc.value == [True,False,True], "should be [True,False,True]"

In [37]:
def test_pbc_value(correct_value = [True,False,True]):
    
    print("Testing pbc value...")
    
    structure = StructureData(
        ase=atoms,
        properties={
                'pbc': {'value':correct_value},
                },
        )

    assert structure.properties.pbc.value == correct_value, f"value should be {correct_value}"
    assert isinstance(structure.properties.pbc.value, type(correct_value)), "type of the value should be {type(correct_value)}"
    
    print("... Testing pbc value passed without errors.")

In [38]:
test_pbc_value()

Testing pbc value...
... Testing pbc value passed without errors.
